In [ ]:
# pepsi.evaluation.ipynb
# by: Tim Lucas Halt (MatrNr. 6682645)

# Vorbereitung
Zunächst werden die notwendigen Bibliotheken installiert und importiert. Möglicherweise ist ein Neustart des Kernels erforderlich.

In [ ]:
%pip install scipy
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install tensorflow
%pip install sweetviz

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sweetviz as sw
import seaborn as sns
sns.set()

Laden des Datensatzes

In [ ]:
# load the schiller text
path = "https://raw.githubusercontent.com/schutera/tiny_schiller/main/tiny_schiller.txt"
filepath = tf.keras.utils.get_file("schiller.txt", path)
with open(filepath) as f:
    schiller_text = f.read()

# Preprocessing

In [32]:
# generate tokens on charakter level
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(schiller_text)

max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
[encoded]=np.array(tokenizer.texts_to_sequences([schiller_text]))-1

train_size=dataset_size*80//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps=100
window_length=n_steps+1
dataset=dataset.repeat().window(window_length,shift=1,drop_remainder=True)

dataset=dataset.flat_map(lambda window: window.batch(window_length))

batch_size=1024
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
dataset=dataset.map(lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch))
dataset=dataset.prefetch(1)

# Definition und Training des Netzes
Um Zusammenhänge zwischen verschiedenen Worten zu bewahren wären LSTM und GRU Layer verwendet.

In [50]:
model=keras.models.Sequential([
    keras.layers.LSTM(256, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
history=model.fit(dataset,
                  steps_per_epoch=train_size/batch_size,
                  epochs=10)

# Textgenerierung
Zuerst werden die Functionen definiert, die die den Text verfollständigen.

In [45]:
def preprocess(texts):
    X=np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X,max_id)

def next_char(text,temperature=1,model=model):
    X_new=preprocess([text])
    y_proba=model.predict(X_new)[0,-1:,:]
    rescaled_logits=tf.math.log(y_proba)/temperature
    char_id=tf.random.categorical(rescaled_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text,n_chars=50,temperature=0.1):
    for _ in range(n_chars):
        text+=next_char(text,temperature)
    return text

Anschließend kann ein Text, ausgehend von einem vordefiniertem Wort/Satz generiert werden.

In [ ]:
def complete_text(text,n_chars=50,temperature=0.01):
    for _ in range(n_chars):
        text+=next_char(text,temperature,model)
    return text

print("Some predicted texts for word 'shakespeare' are as follows:\n ")
print(complete_text('shakespeare'))

Scheinbar hat das drunkenSchiller Modell, das in dem Git-Repository gespeichert ist eine Vorliebe für Geschichte...

In [ ]:
# load a saved drunkenschiller
model_name = '../Code/model/drunkenSchiller.h5' 
model_reloaded = tf.keras.models.load_model(model_name)

def complete_text(text,n_chars=50,temperature=0.01):
    for _ in range(n_chars):
        text+=next_char(text,temperature,model_reloaded)
    return text

print("Some predicted texts for word 'Geschichte' are as follows:\n ")
print(complete_text('Geschichte'))